In [ ]:
!pip install "pyautogen[gemini]~=0.2.0b4"

In [ ]:
!pip install https://github.com/microsoft/autogen/archive/gemini.zip

In [ ]:
pip install "google-generativeai" "pydash" "pillow"

In [29]:
import requests
import json
import pdb
import os
import re

from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent

from autogen.agentchat.contrib.img_utils import get_image_data, _to_pil
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

from termcolor import colored
import random

from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

In [30]:
with open('../keys.json') as f:
    keys = json.load(f)

In [31]:
assistant = AssistantAgent("assistant", 
                           llm_config={"temperature":0,"config_list": [{'model': 'gemini-pro',
  'api_key': keys["GOOGLE_API_KEY"],
  'api_type': 'google'}], "cache_seed": None}, 
                           max_consecutive_auto_reply=3)

In [32]:
print(assistant.system_message)

You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feed

In [33]:
user_proxy = UserProxyAgent("user_proxy", 
                            code_execution_config={"work_dir": "coding", "use_docker": False}, 
                            human_input_mode="NEVER", 
                            is_termination_msg = lambda x: content_str(x.get("content")).find("TERMINATE") >= 0,
                            llm_config={"temperature":0,"config_list": [{'model': 'gemini-pro',
  'api_key': keys["GOOGLE_API_KEY"],
  'api_type': 'google'}], "cache_seed": None})

In [34]:
user_proxy.initiate_chat(assistant, message="provide code to print hello world",clear_history=True)

user_proxy (to assistant):

provide code to print hello world

--------------------------------------------------------------------------------


Model gemini-pro not found. Using cl100k_base encoding.


assistant (to user_proxy):

```python
# filename: hello_world.py
print("hello world")
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
hello world


--------------------------------------------------------------------------------


Model gemini-pro not found. Using cl100k_base encoding.


assistant (to user_proxy):

Great! You have successfully printed "hello world" using Python.

TERMINATE

--------------------------------------------------------------------------------


In [36]:
user_proxy.chat_messages

defaultdict(list,
            {<autogen.agentchat.assistant_agent.AssistantAgent at 0x7ad3b43bb880>: [{'content': 'provide code to print hello world',
               'role': 'assistant'},
              {'content': '```python\n# filename: hello_world.py\nprint("hello world")\n```',
               'role': 'user'},
              {'content': 'exitcode: 0 (execution succeeded)\nCode output: \nhello world\n',
               'role': 'assistant'},
              {'content': 'Great! You have successfully printed "hello world" using Python.\n\nTERMINATE',
               'role': 'user'}]})

In [37]:
user_proxy.last_message()

{'content': 'Great! You have successfully printed "hello world" using Python.\n\nTERMINATE',
 'role': 'user'}

In [27]:
from autogen import OpenAIWrapper
client = OpenAIWrapper(config_list=[
        {
            'model': 'gemini-pro',
            'api_key': keys["GOOGLE_API_KEY"],
            'api_type': 'google'
        }
    ])
response = client.create(messages=[{"role": "user", "content": "what is the most likely word to come after this?"}],temperature=0)
print(client.extract_text_or_completion_object(response))

Model gemini-pro not found. Using cl100k_base encoding.


['The most likely word to come after what depends on the context. For example, if the preceding words are "I wonder", the most likely word to come after is "what". If the preceding words are "I need", the most likely word to come after is "help". And if the preceding words are "The cat sat on the", the most likely word to come after is "mat".\n\nWithout any context, it is impossible to say what the most likely word to come after is. However, some of the most common words in the English language include "the", "of", "and", "a", "to", "in", "is", "you", "that", and "it". These words are likely to appear after many different words or phrases.\n\nTo provide a more specific answer, I would need more information about the context in which the word is being used.']


In [14]:
response

ChatCompletion(id='822', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello, World!', role='assistant', function_call=None, tool_calls=None))], created=1706584818838, model='gemini-pro', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=10, total_tokens=14), cost=0, config_id=0, pass_filter=True)